# CPSC 340 Lecture 33: Deep learning software

## Programming languages

- In ML we use a variety of programming languages: Python, Java, R, C++, Matlab, many more. 
  - In CPSC 340 we used Python so I'll mainly stick to Python packages in this lecture.
  - Most deep learning packages are Python-based in any case.

- A popular open-source library is [scikit-learn](http://scikit-learn.org/stable/), which we've used a few times in the course. 

- In Assignment 6 you'll use sklearn's [neural network implementation](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html). By the way, this was [written by your TA, Issam](https://github.com/scikit-learn/scikit-learn/blob/14031f6/sklearn/neural_network/multilayer_perceptron.py#L4).

## Deep learning software

There's been a lot of software released lately to take care of this for you. Some big players are:

| Name   |  Host language  | Released |  Comments | 
|--------|-------------|---------------|----------|
| [Theano](http://deeplearning.net/software/theano/) | Python | 2007 | From U. de Montréal |
| [Torch](http://torch.ch) | Lua | 2002 | Used at Facebook |
| [PyTorch](http://pytorch.org) | Python | 2017 | Automatic differentiation through arbitrary code like [Autograd](https://github.com/HIPS/autograd)
| [TensorFlow](https://www.tensorflow.org) | Python | 2015 | Created by Google for both prototyping and production
| [Keras](https://keras.io) | Python | 2015 | A front-end on top of Theano or TensorFlow, soon to be more integrated |
| [Caffe](http://caffe.berkeleyvision.org) | Executable with Python wrapper | 2014 | Specifically for convnets (see Lectures 31-32), from UC Berkeley

- There are many others of course. See for example [this Wikipedia article](https://en.wikipedia.org/wiki/Comparison_of_deep_learning_software).
- From the table above, we can see there have been a lot of new packages released recently.

## GPUs

- Part of the recent progress in deep learning has been due to the computational power of GPUs.
- GPUs were originally designed for graphics, which requires a lot of _matrix multiplication_.
  - This is exactly what we need for neural networks.
- The leader is NVIDIA, an American company, and their GPU programming language is called CUDA.
  - Luckily, we rarely need to write CUDA code anymore, as there are intermediate layers of software.

## Cloud computing

- I'm guessing few/none of the laptops in the room have a CUDA-capable NVIDIA GPU capable of fast training. 
  - My Macbook Air certainly isn't good for much in terms of computation.
- Luckily, the advent of cloud computing platforms like Amazon's EC2 gives us easy access to computing resources.
- Amazon also offers [Amazon Machine Images](https://en.wikipedia.org/wiki/Amazon_Machine_Image) (AMIs) which simplify things further. 
  - This is like a virtual machine and comes with relevant software installed, etc.
  - The AMI costs a bit extra on top of the EC2 costs (say 10%)
  - If you were a big company you would probably create your own, but these public AMIs are great for prototyping.
  - If you're interested in containerization, [Docker](https://www.docker.com/) is gaining a lot of popularity.

## Demos: loading the data

The data is built in to Keras, so we just access it for convenience. If not present already it is automatically downloaded.

In [9]:
from keras.datasets import mnist

# the data, shuffled and split between train and test sets
(X_train, y_train_cat), (X_test, y_test_cat) = mnist.load_data()

img_dim = (28,28) 
img_size = img_dim[0]*img_dim[1]
num_classes = 10

X_train = X_train.astype('float32')
X_test  = X_test.astype('float32')
X_train /= 255
X_test  /= 255
X_train_flat = X_train.reshape(60000, img_size)
X_test_flat  = X_test.reshape(10000, img_size)
X_train = X_train[...,None] # add 4th dimension, needed later for convnets
X_test  = X_test[...,None]

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train_cat, num_classes)
y_test = np_utils.to_categorical(y_test_cat, num_classes)

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


## Demo 1: scikit-learn on my laptop

See documentation for the [classifier](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) and [regressor](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html).


In [ ]:
from sklearn.neural_network import MLPClassifier

In [24]:
%%timeit -n 1 -r 1

nn = MLPClassifier(hidden_layer_sizes=(100,100), max_iter=10, batch_size=128)
nn.fit(X_train_flat, y_train_cat)

score = nn.score(X_train_flat, y_train_cat)
print('Train accuracy:', score)

score = nn.score(X_test_flat, y_test_cat)
print('Test accuracy:', score)

//anaconda/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


Train accuracy: 0.995783333333
Test accuracy: 0.9761
1 loop, best of 1: 30.2 s per loop


## Demo 2: Keras/TensorFlow on my laptop


### Fully-connected net

Attribution: the code below is adapted from the [Keras MNIST example](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py), which is under the [MIT license](https://github.com/fchollet/keras/blob/master/LICENSE).


** Imports and preprocessing **

In [16]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils

** Model definition **

Here we need to specify the input and output size in advance (unlike sklearn) because the model is first _compiled_.

In [28]:
model = Sequential()
model.add(Dense(100, input_shape=(X_train_flat.shape[1],), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

** Training and evaluation **

In [29]:
%%timeit -n 1 -r 1

history = model.fit(X_train_flat, y_train,
                    batch_size=128, 
                    nb_epoch=10,
                    verbose=0)

score = model.evaluate(X_train_flat, y_train, verbose=0)
print('Train accuracy:', score[1])

score = model.evaluate(X_test_flat, y_test, verbose=0)
print('Test accuracy:', score[1])

Train accuracy: 0.993483333333
Test accuracy: 0.9769
1 loop, best of 1: 40 s per loop


### Convolutional net

Attribution: the code below is adapted from [Deep Learning with Python](https://machinelearningmastery.com/deep-learning-with-python2/) with permission from the author.

** Model definition **

In [28]:
model = Sequential()
model.add(Convolution2D(32, 5, 5, input_shape=img_dim+(1,), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

** Training and evaluation **

In [31]:
%%timeit -n 1 -r 1

history = model.fit(X_train, y_train,
                    batch_size=128, nb_epoch=1,
                    verbose=0, validation_data=(X_test, y_test))

score = model.evaluate(X_train, y_train, verbose=0)
print('Train accuracy:', score[1])

score = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', score[1])

Train accuracy: 0.99475
Test accuracy: 0.9893
1 loop, best of 1: 1min 24s per loop


## Demo 3: Keras/TensorFlow on EC2 (with GPU)

- I'll now attempt to demo Keras/TensorFlow on EC2 "from scratch".
  - By this I mean I haven't done much of the work in advance.
  - You should be able to follow along if you already have an AWS account and you're willing to pay a few dollars.
- This is probably a bad idea. I'll give the demo a 50% chance of working.
  
Here are the steps I'm about to do:

1. (already done) sign into [AWS](https://aws.amazon.com).
2. Go to the [AWS console](https://console.aws.amazon.com) and select EC2.
3. Select "Key Pairs" on the left and create a new Key Pair.

2. Visit the site of the [AMI I'm using](https://aws.amazon.com/marketplace/pp/B01EYKBEQ0).
3. Press "Continue"
4. Select US West (Oregon) as my [EC2 region](http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/using-regions-availability-zones.html), and p2.xlarge as my [EC2 instance type](https://aws.amazon.com/ec2/instance-types/).
5. Launch with 1-Click. The cost is \$1/hour.
5. Go back to the EC2 console and click Instances on the left side. 
6. Wait until the EC2 console lists the Instance State as "running"...
7. Right-click the instance in the console, and press Connect. Follow the instructions there. 
  - change permissions on key
  - `ssh` into the instance
8. Run `watch -n 1 nvidia-smi` so I can monitor the GPU. You'll see it's a [Tesla K80](http://www.nvidia.com/object/tesla-k80.html), which is [not cheap](https://www.amazon.com/HP-J0G95A-NVIDIA-Tesla-K80/dp/B00TWFEIWA). It starts around body temperature.
8. And now... go back to the console and get the public IP of the instance.
9. Point my browser to `http://{EC2 Instance Public IP}:8888`
10. Grab the instance id from the EC2 console: this is the password.
11. Create a notebook, paste in the above code. And run!
12. Go back to the terminal and see that the GPU is being utilized. Is the temperature going up?
13. **IMPORTANT** Go back to the EC2 console, right-click on the instance, Instance State --> Terminate.

Compared to my laptop, I observed a speedup of about **10x**. This translates into better models when training time is the limiting factor.

## Aside: spot instances

- EC2 offers both "dedicated instances" and ["spot instances"](https://aws.amazon.com/ec2/spot/). 
- Spot instances and much cheaper but may be interrupted if the "spot price" goes above your maximum price. 
  - For example, our instance is about \$1/hour and a spot instance would be about \$0.30/hour
- In general you should always use spot instances when running experiments because of price.
  - You'd want dedicated instances if you were _deploying_ your trained model in an app, though!
- I am not using a spot instance in this demo because they aren't supported with the "1-Click Launch" feature of this AMI.
  - But if I wasn't doing a demo then I'd launch a spot instance myself, through the EC2 console.

## Aside: Amazon educate

- Amazon has a program that gives [free credit for students](https://aws.amazon.com/education/awseducate/). 
- However, because UBC uses `@alumni.ubc.ca` email addresses for current students, the automated system tends to reject applications from UBC students.
- If you have a CS ugrad account you can probably use it (but I think these are disappearing next year?).
- We are working on this, maybe it will be solved by next year...

## Final advice: terminate your EC2 instances!!!

- This may be the most important advice I give you in CPSC 340 ;)
- If you forget to do this, you can be stuck with a big bill from Amazon.
- You can do this from the EC2 console.